Itay Koren

In [3]:
pip install psycopg2 -U

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import reflection
import psycopg2

### Access to the DB <a class="anchor" id="chapter2"></a>

In [5]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-fc-barcelona-20-21-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})
inspector = reflection.Inspector.from_engine(engine)


<ipython-input-5-ba3530ab46e4>:14: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = reflection.Inspector.from_engine(engine)


#  Table Queries <a class="anchor" id="chapter3"></a>

function that takes a query and return dataframe for general use

In [6]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)
    

In [7]:
inspector.get_table_names()

['squad',
 'games_played',
 'league_stats',
 'champions_stats',
 'cup_stats',
 'supercup_stats']

1. How many users got paid?

user_id
payout


squad table:

        player_id: player's id

        first_name: player's first name

        last_name: player's last name

        position: position in the field - gk,df,mf,fw


games_played table:

        player_id: player's id

        league: league appearances

        champions: champions appearances

        cup: cup appearances

        supercup: supercup appearances
league_stats table:

        player_id: player's id

        league_goals: goals in league

        league_assits: assits in league

        league_yellow: yellow cards in league

        league_red: red cards in league

        league_minutes: minutes played in league

champions_stats table:

        player_id: player's id

        champions_goals: goals in champions

        champions_assits: assits in champions

        champions_yellow: yellow cards in champions

        champions_red: red cards in champions

        champions_minutes: minutes played in champions

cup_stats table:

        player_id: player's id

        cup_goals: goals in cup

        cup_assits: assits in cup

        cup_yellow: yellow cards in cup

        cup_red: red cards in cup

        cup_minutes: minutes played in cup
        
supercup_stats table:

        player_id: player's id

        supercup_goals: goals in supercup

        supercup_assits: assits in supercup

        supercup_yellow: yellow cards in supercup

        supercup_red: red cards in supercup

        supercup_minutes: minutes played in supercup

1. what is the diff in percentage between avg yellow cards per game in the cup and champions between df and fw players (meaning you first need to calculate sum of yellow cards for relevant position's players per game and then divide each position’s total yellow cards by the total of both positions)


In [8]:
query = '''
select 
    position,
    sum(cup_yellow + champions_yellow) / sum(champions + cup) :: real as ratio
from 
    squad s
    left join cup_stats cup
    on s.player_id = cup.player_id 
    left join champions_stats champ
    on champ.player_id = s.player_id
    left join games_played games
    on games.player_id = s.player_id
where
    position = 'df' or position = 'fw'
GROUP BY 
    position  
'''

queryResult(query)

,position,ratio
0,fw,0.109375
1,df,0.160000



2. Per player calculate the avg percentage of games played.


In [9]:
query = '''
select 
    player_id,
    (league + cup + champions + supercup)/54 ::real as avg_played_games 
from 
    games_played
'''

queryResult(query)

,player_id,avg_played_games
0,1,0.777778
1,2,0.222222
2,3,0.000000
3,4,0.000000
4,5,0.333333
5,6,0.759259
6,7,0.425926
7,8,0.611111
8,9,0.888889
9,10,0.296296



3. for each competition return a list of palyers that didn't get any yellow or red cards


In [10]:
query = '''
select 
    case when cup_red = 0 and cup_yellow = 0
    then cup.player_id
    else -1
    end as player_cup_no_red_yellow,

    case when champions_red = 0 and champions_yellow = 0
    then champ.player_id
    else -1
    end as player_champ_no_red_yellow,

    case when supercup_red = 0 and supercup_yellow = 0
    then scup.player_id
    else -1
    end as player_scup_no_red_no_yellow,

    case when league_red = 0 and league_yellow = 0
    then league.player_id
    else -1
    end as player_league_no_red_yellow
from 
    cup_stats cup
    
    left join champions_stats champ
    on champ.player_id = cup.player_id

    left join  supercup_stats scup
    on scup.player_id = cup.player_id

    left join league_stats as league
    on league.player_id = cup.player_id
where
    (champions_yellow = 0 and champions_red = 0)
    or (cup_yellow = 0 and cup_red =0)
    or (supercup_yellow = 0 and supercup_red = 0)
    or (league_yellow = 0 and league_red = 0)
'''

queryResult(query)

,player_cup_no_red_yellow,player_champ_no_red_yellow,player_scup_no_red_no_yellow,player_league_no_red_yellow
0,1,1,1,-1
1,2,2,2,2
2,3,3,3,3
3,4,4,4,4
4,5,-1,5,-1
5,6,6,6,-1
6,7,-1,7,-1
7,8,8,8,-1
8,9,-1,-1,-1
9,10,-1,10,-1



4. from each position please return the player that played the most in the cup game and had at least one yellow card


In [11]:
query = '''
select
    position,
    player_id,
    rank,
    cup_minutes
from
    (select 
        cup.player_id,
        position,
        rank() over(partition by  position order by cup_minutes desc) as rank,
        cup_minutes
    from 
        cup_stats cup
        left join squad on squad.player_id = cup.player_id
    where
        cup_yellow > 0) as subq
where
    rank = 1
'''

queryResult(query)

,position,player_id,rank,cup_minutes
0,df,11,1,465
1,fw,29,1,510
2,mf,17,1,162



5. What is the average of yellow cards per position? (The share of each position)


In [12]:
query = '''
select 
    position,
    sum(cup_yellow + champions_yellow + supercup_yellow + league_yellow),
    sum_yellow,
    sum(cup_yellow + champions_yellow + supercup_yellow + league_yellow) / sum_yellow :: real as share_yellow
from 
    (
    select
        position,
        cup_yellow,
        champions_yellow,
        supercup_yellow,
        league_yellow,
        sum(cup_yellow + champions_yellow + supercup_yellow + league_yellow) over() as sum_yellow
    from 
        squad s
        
        left join cup_stats cup
        on s.player_id = cup.player_id 
        
        left join champions_stats champ
        on champ.player_id = s.player_id
        
        left join supercup_stats as scup
        on scup.player_id = s.player_id

        left join league_stats as league
        on league.player_id = cup.player_id
        ) as subq
where
    position = 'df' or position = 'fw'
group by
    position, sum_yellow

'''

queryResult(query)

,position,sum,sum_yellow,share_yellow
0,fw,23,94,0.244681
1,df,46,94,0.489362



6. Who are the top 3 players with the best assists per minute record? (How often they make an assist – 3 assists in 900 minutes means assist every 300 minutes)


In [13]:
query = '''
select 
    s.player_id,
    sum(cup.cup_minutes + supercup_minutes + champions_minutes + league_minutes) as minutes,
    sum(cup_assists + supercup_assists + champions_assists + league_assists) as assists,
    case when sum(cup.cup_minutes + supercup_minutes + champions_minutes + league_minutes) = 0
    then 0
    else 100 * sum(cup_assists + supercup_assists + champions_assists + league_assists) / sum(cup.cup_minutes + supercup_minutes + champions_minutes + league_minutes) ::real
    end as assists_per_minute
from 
    squad s

    left join cup_stats cup
    on s.player_id = cup.player_id 

    left join champions_stats champ
    on champ.player_id = s.player_id

    left join supercup_stats scup
    on scup.player_id = s.player_id

    left join league_stats as league
    on league.player_id = cup.player_id
where
    position = 'df' or position = 'fw'
group by 
    s.player_id  
order by
    assists_per_minute
'''

queryResult(query)

,player_id,minutes,assists,assists_per_minute
0,23,36,0,0.000000
1,7,1923,0,0.000000
2,9,3698,0,0.000000
3,10,827,0,0.000000
4,26,0,0,0.000000
5,6,2745,1,0.036430
6,8,2231,1,0.044823
7,11,2796,3,0.107296
8,5,864,1,0.115741
9,25,1309,2,0.152788



7. Each game consists of 90 minutes what is the share of each player playing time in the league and cup from his played games?
Example: if a player played 1000 minutes and took part in 12 games then his share is 92.59% (1000 / (12*90) )


In [15]:
query = '''
select 
    cup.player_id,
    sum(cup_minutes + league_minutes) as player_total_min,
    sum(cup + league) as total_appearance,
    sum(cup + league) * 90 as games_total_min,
    case when sum(cup+league) = 0
    then 0
    else sum(cup_minutes + league_minutes) / (sum(cup+league)*90) :: real
    end as min_per_appearance
from 
    cup_stats cup

    left join league_stats as league
    on league.player_id = cup.player_id

    left join games_played as gmplayed
    on gmplayed.player_id = cup.player_id
group by
    cup.player_id
'''

queryResult(query)

,player_id,player_total_min,total_appearance,games_total_min,min_per_appearance
0,1,3210,35,3150,1.019048
1,2,840,9,810,1.037037
2,3,0,0,0,0.000000
3,4,0,0,0,0.000000
4,5,605,11,990,0.611111
5,6,2113,33,2970,0.711448
6,7,1687,20,1800,0.937222
7,8,1844,28,2520,0.731746
8,9,2911,38,3420,0.851170
9,10,792,15,1350,0.586667





8. A forward (fw) is considered to have a good season if he played at least 1500 minutes , took part in at least 18 games and got a minimum of 9 goals + assists.

How many forwards like that do we have?

In [24]:
query = '''
select 
    count(s.player_id)
from 
    squad s

    left join cup_stats as cup
    on cup.player_id = s.player_id

    left join league_stats as league
    on league.player_id = cup.player_id

    left join champions_stats as champ
    on champ.player_id = s.player_id

    left join supercup_stats as scup
    on scup.player_id = s.player_id

    left join games_played as gmplayed
    on gmplayed.player_id = cup.player_id
where
    position = 'fw'    
having
    sum(cup_minutes + league_minutes + champions_minutes + supercup_minutes) >=1500
    and
    sum(cup_assists + cup_goals + league_assists + league_goals + champions_assists + champions_goals + supercup_assists + supercup_goals) >=9
    and
    sum(cup + league + champions + supercup) >= 18
'''

queryResult(query)

,count
0,8
